# Support Vector Machines

A *support vector machine* (SVM) is a very powerful & versatile machine learning model, capable of performing linear or nonlinear classification, regression, & even outlier detection. It is one of the most popular models in machine learning, & any one interested in machine learning should have in their toolbox. SVMs are particularly well suited for classification of complex but small-or medium-sized datasets.

---

# Linear SVM Classification

The fundamental idea behind SVMs is best explained with pictures. 

<img src = "Images/Large Margin Classification.png" alt = "Alternative text" width = "700" style = "margin:auto"/>

The above figure shows part of the iris dataset. The two classes can clearly be separated easily with a straight line (they are *linearly separable*). The left plot shows the decision boundaries of three possible linear classifiers. The model whose decision boundary is represented by the dashed line that it does not even separate the classes properly. The other two models work perfectly on this training set, but their decision boundaries come so close to the instances that these models will probably not perform well on new instances. In contrast, the solid line in the plot on the right represents the decision boundary of an SVM classifier; this line not only separates the two classes but also stays as far from the closest training instances as possible. You can think of an SVN classifier as fitting the widest possible street (represented by the parallel dashed lines) between the classes.. This is called *large margin classification*.

Notice that adding more training instances "off the street" will not affect the decision boundary at all: it is fully determined (or "supported") by the instances located on the edge of the street. These instances are called *support vectors* (they are circled in the above figure).

## Soft Margin Classification

If we strictly impose that all instances be off the street & on the right side, this is called *hard margin classification*. There are two main issues with hard margin classification. First, it only works if the data is linearly separable, & second, it is quite sensitive to outliers. The below figure shows the iris dataset with just one additional outlier: on the left, it is impossible to find a hard margin, & on the right, the decision boundary ends up very different from the one we saw int he figure above without the outlier, & it will probably not generalise as well.

<img src = "Images/Hard Margin Sensitivity to Outliers.png" width = "700" style = "margin:auto"/>

To avoid these issues, it is preferable to use a more flexible model. The objective is to find a good balance between keeping the street as large as possibel & limiting the *margin violations* (i.e., instances that end up in the middle of the street or even on the wrong side). This is called *soft margin classification*.

In scikit-learn's SVM classes, you can control this balance using the `C` hyperparameter: a smaller `C` value leads to a wider street but more margin violations. The below figure shows the decision boundaries & margins of two soft margin svm classifiers on a nonlinearly separable dataset.

<img src = "Images/Large Margin vs Smaller Margin.png" width = "700" style = "margin:auto"/>

On the left, using a low `C` value, the margin is quite large, but many instances end up on the street. On the right using a high `C` value, the classifier makes fewer margin violations but ends up with a smaller margin. However, it seems likely that the first classifier will generalise better: in fact even on this training set it makes fewer prediction errors, since most of the margin violations are actually on the correct side of the decision boundary.

The following scikit-learn code loads the iris dataset, scales the features, & then trains a linear SVM model (using the `LinearSVC` class with `C = 1` & the *hinge loss* function) to detect Iris-Virginica flowers. The resulting model is represented in the left diagram of the above figure.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

svm_classifier = Pipeline([("scaler", StandardScaler()),
                           ("linear_svc", LinearSVC(C = 1, loss = "hinge"))])
svm_classifier.fit(X, y)

/Users/jiehengyu/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

Then, as usual, you can use the model to make predictions:

In [2]:
svm_classifier.predict([[5.5, 1.7]])

array([1.])

Alternatively, you could use the `SVC` class using `SVC(kernel = "linear", C = 1)`, but it is much slower, especially with large training sets, so it is not recommended. Another option is to use the `SGDClassifier` class, with `SGDClassifier(loss = "hinge", alpha = 1/(m * C))`. This applies regular stochastic gradient descent to train a linear SVM classifier. It does not converge as fast as the `LinearSVC` class, but it can be useful to handle huge datasets that do not fit in memory (out-of-core training), or to handle online classification tasks.

---

# Nonlinear SVM Classification

Although linear SVM classifiers are efficient & work surprisingly well in many cases, many datasets are not even close to being linearly separable. One approach to handling nonlinear datasets is to add more features, such as polynomial features; in some cases, this can result in a linearly separable dataset. Consider the figure below, it represents a simple dataset with just one feature $x_1$. This dataset is not linearly separable, as you can see. But if you add a second feature $x_2 = (x_1)^2$, the resulting 2D dataset is perfectly linearly separable.

<img src = "Images/Add Features to Make Data Linearly Separable.png" width = "600" style = "margin:auto"/>

To implement this idea using scikit-learn, you can create a `Pipeline` containing a `PolynomialFeatures` transformer,followed by a `StandardScaler` & a `LinearSVC`. Let's test this on the moons dataset: this is toy dataset for binary classification in which the data points are shaped as two interleaving half circles. You can generate this dataset using the `make_moons()` function:

In [6]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

polynomial_SVM_classifier = Pipeline([("polyfeature", PolynomialFeatures(degree = 3)),
                                      ("scaler", StandardScaler()),
                                      ("svmclassifier", LinearSVC(C = 10, loss = "hinge"))])
polynomial_SVM_classifier.fit(X, y)

Pipeline(steps=[('polyfeature', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svmclassifier', LinearSVC(C=10, loss='hinge'))])

<img src = "Images/Linear SVM Classifier using Polynomial Features.png" width = "400" style = "margin:auto"/>

## Polynomial Kernel

Adding polynomial features is simple to implement & can work great with all sorts of machine learning algorithms (not just SVMs), but at a low polynomial degree it cannot deal with very complex datasets, & with a high polynomial degree it creates a huge number of features, making the model too slow.

Fortunately, when using SVMs, you can apply an almost miraculous mathematical technique called the *kernel trick* (it is explained in a moment). It makes it possible to get the same result as if you added many polynomial features, even with very high-degree polynomials, without actually having to add them. So there is no combinatorial explosion of the number of features since you don't actually add any features. This trick is implemented by the `SVC` class. Let's test it on the moons dataset:

In [8]:
from sklearn.svm import SVC

poly_kernel_svm_classifier = Pipeline([("scaler", StandardScaler()),
                                       ("svmclassifier", SVC(kernel = "poly", degree = 3, coef0 = 1, C = 5))])
poly_kernel_svm_classifier.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svmclassifier', SVC(C=5, coef0=1, kernel='poly'))])

This code trains an SVM classifier using a $3^{rd}$-degree polynomial kernel. It is represented in the below figure.

<img src = "Images/SVM Classifiers with Polynomial Kernel.png" width = "600" style = "margin:auto"/>

On the right is another SVM classifier using a $10^{th}$-degree polynomial kernel. Obviously, if your model is overfitting, you might want to reduce the polynomial degree. Conversely, if it is underfitting, you can try increasing it. The hyperparameter `coef0` controls how much the model is influenced by high-degree polynomials versus low-degree polynomials

## Adding Similarity Features

Another technique to tackle nonlinear problems is to add features computed using a *similarity function* that measure how much each instance resembles a particular *landmark*. For example, let's take the one-dimensional dataset discuessed earlier & add two landmarks to it at $X_1 = -2$ & $x_1 = 1$. Next, let's define the similarity function to be the Gaussian *Radial Basis Function* (RBF) with $\gamma = 0.3$

$$\phi_\gamma(x, l) = e^{(-\gamma||x - l||^2)}$$

<img src = "Images/Similarity Features Using Gaussian RBF.png" width = "700" style = "margin:auto"/>

You may wonder how to select the landmarks. The simplest approach is to create a landmark at the location of each & every instance in the dataset. This creates many dimensions & thus increases the chances that the transformed training set will be linearly separable. The downside is that a training set with *m* instances & *n* features get transformed into a training set with *m* instances & *m* features (assuming you drop the original features). If your training set is very large, you end up with an equally large number of features.

## Gaussian RBF Kernel

Just like the polynomial features method, the similarity features method can be useful with any machine learning algorithm, but it may be computationally expensive to compute all the additional features, especially on large training set. However, once again the kernel trick does its SVM magic: it makes it possible to obtain a similar result as if you had added many similarity features, without actually having to add them. let's try the gaussian RBF kernel using the `SVC` class:

In [9]:
rbf_kernel_svm_classifier = Pipeline([("scaler", StandardScaler()),
                                      ("svmclassifier", SVC(kernel = "rbf", gamma = 5, C = 0.001))])
rbf_kernel_svm_classifier.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svmclassifier', SVC(C=0.001, gamma=5))])

This model is represented on the bottom left below.

<img src = "Images/SVM Classifier with RBF Kernel.png" width = "600" style = "margin:auto"/>

The other plots show models trained with different values of hyperparameters `gamma` ($\gamma$) & `C`. Increasing `gamma` makes the bell-shape curve narrower, & as a result, each instance's range of influence is smaller: the decision boundary ends up being more irregular, wiggling around individual instances. Conversely, a small `gamma` value makes the bell-shaped curve wider, so instances have a larger range of influence, & the decision boundary ends up smoother. So $\gamma$ acts like a regularisation hyperparameter: if your model is overfitting, you should reduce it; & if it is underfitting, you should increase it (similar to the `C` hyperparameter). 

Other kernels exist but are used much more rarely. For example, some kernels are specialised for specific data structures. *String kernels* are sometimes used when classifying text documents or DNA sequences (e.g., using the *string subsequence kernel* or kernels based on the *Levenshtein distance*).

## Computation Complexity

The `LinearSVC` class is based on the *liblinear* library, which implements an optimised algorithm for linear SVMs. It does not support the kernel trick, but it scales amlmost linearly with the number of training instances & the number of features: its training time complexity is roughly $O(m * n)$.

The algorithm takes longer if you require a very high precision. This is controlled by the tolerance hyperparameter $\epsilon$ (called `tol` in scikit-learn). In most classification tasks, the default tolerance is fine.

The `SVC` class is based on the *libsvm* library, which implements an algorithm that supports the kernel trick. The training time complexity is usually between $O(m^2 * n)$ & $O(m^3 * n)$. Unfortunately, this means that it gets dreadfully slow when the number of training instnaces gets large (e.g., hundreds of thousands of instances). This algorithm is perfect for complex but small or medium training sets. However, it scales well with the number of features, especially with *sparse features* (i.e., when each instance has few nonzero features). In this case, the algorithm scales roughly with the average number of nonzero features per instance. This table compared scikit-learn's SVM classification classes.

|**Class**|**Time Complexity**|**Out-of-core Support**|**Scaling Required**|**Kernel Trick**|
|:----:|:----:|:----:|:----:|:----:|
|LinearSVC|$O(m * n)$|No|Yes|No|
|SGDClassifier|$O(m * n)$|Yes|Yes|No|
|SVC|$O(m^2 * n)$ to $O(m^3 * n)$|No|Yes|Yes|

---

# SVM Regression

As we mentioned earlier, the SVM algorithm is quite versatile: not only does it support linear & nonlinear classification, but it also supports linear & nonlinear regression. The trick is to reverse the objective: instead of trying to fit the large possible street between two classes while limiting margin violations, SVM Regression tries to fit as many instances as possible on the street while limiting margin violations (i.e, instnace *off* the street). The width of the street is controlled by a hyperparameter $\epsilon$. The below figure shows two linear SVM regression models trained on some random linear data, one with a large margin ($\epsilon = 1.5$) & the other with a small marge ($\epsilon = 0.5$).

<img src = "Images/SVM Regression.png" width = "500" style = "margin:auto"/>

Adding more training instances within the margin does not affect the model's predictions; thus, the model is said to be $\epsilon$-*insensitive*.

You can use scikit-learn's `LinearSVR` class to perform linear SVM regression. The following code produces the model represented on the left of the above figure (the training data should be scaled & centered first):

In [11]:
from sklearn.svm import LinearSVR

svmreg = LinearSVR(epsilon = 1.5)
svmreg.fit(X, y)

LinearSVR(epsilon=1.5)

To tackle nonlinear regression tasks, you can use a kernelised SVM model. For example, the following figure shows SVM regression on a random quadratic training set, using a $2^{nd}$-degree polynomial kernel. There is little regularisation on the left plot (i.e., a large `C` value), & much more regularisation on the right plot (i.e., a small `C` value).

<img src = "Images/SVM Regression with 2nd Degree Polynomial Kernel.png" width = "600" style = "margin:auto"/>

The following code produces the model represented on the left of the above figure using scikit-learn's `SVR` class (whuch supports the kernel trick). The `SVR` class is the regression equivalent of the `SVC` class, & the `LinearSVR` class is the regression equivalent of the `LinearSVC` class. The `LinearSVR` class scales linearly with the size of the training set (just like the `LinearSVC` class), while the `SVR` class gets much too slow when the training set grows large (just like the `SVC` class).

In [12]:
from sklearn.svm import SVR

svm_polyreg = SVR(kernel = "poly", degree = 2, C = 100, epsilon = 0.1)
svm_polyreg.fit(X, y)

SVR(C=100, degree=2, kernel='poly')

---

# Under the Hood

Let's see how SVMs make predictions & how their training algorithms work, starting with linear SVM classifiers. In previous lessons, we used the conventions of putting all the model parameters in one vector $\theta$, including the bias term $\theta_0$ &the input feature weights $\theta_1$ to $\theta_n$, & adding a bias input $x_0 = 1$ to all instances. We will use more convenient (& more common) conventions for dealing with SVMs: the bias term will be called *b* & the feature weights vector will be called $w$. No bias feature will be added to the input feature vectors.

## Decision Function & Predictions

The linear SVM classifier model predicts the class of a new instance $x$ by simply computing the decision function $w^tx + b = w_1x_1 + ... + w_nx_n + b$: if the result is positive, the predicted class $\hat{y}$ is the positive class (1), or else it is the negative class (0).

$$\hat{y} = 
\Biggl\{\begin{split}
{0\ if\ w^Tx + b < 0} \\
{1\ if\ w^Tx + b \geq 0} \\
\end{split}$$

The below figure shows the decision function that corresponds to the left diagram of our third figure: it is a two-dimensional plane since this dataset has two features (petal width & petal length). The decision boundary is the set of points where the decision function is equal to 0: it is the intersection of two planes, which is a straight line (represented by the thick solid line.

<img src = "Images/Decision Function Iris Dataset.png" width = "600" style = "margin:auto"/>

The dashed line represent the points where the decision function is equal to 1 or -1: they are parallel & at equal distance to the decision boundary, forming a margin around it. Training a linear SVM classifier means finding the value of $w$ & $b$ that make this margin as wide as possible while avoiding margin violations (hard margin) or limiting them (soft margin).

## Training Objective

Consider the slope of the decision function: it is equal to the norm of the weight vector, $||w||$. If we divide this sloep by 2, the points where the decision function is equal to $\pm 1$ are going to be twice as far way from the decision boundary. In other words, dividing the slope by 2 will multiple the margin by 2. perhaps this is easier to visualise in the below figure. the smaller the weight vector $w$, the larger the margin.

<img src = "Images/Weight Vector Affects Margin.png" width = "600" style = "margin:auto"/>

So we want to minimise $||w||$ to get a large margin. However, if we also want to avoid any margin violation (hard margin), then we need the decision function to be greater than 1 for all positive trainign instances, & lower than -1 for negative training instances. If we define $t^{(i)} = -1$ for negative instances (if $y^{(i)} = 0$) & $t^{(i)} = 1$ for positive instances (if $y^{(i)} = 1$, then ew can express this constraint as $t^{(i)}(w^tx^{(i)} + b) \pm 1$ for all instances.

We can therefore express the hard margine linear SVM classifier objective as the *constrained optimsation problem*.

$$\begin{split}
\underset{w, b}{minimise}\quad \frac{1}{2}w^Tw \\
subject\ to \quad t^{(i)}(w^Tx^{(i)} + b) \geq 1 \quad for\ i = 1, 2, ..., m 
\end{split}$$

TO get the soft margin objective, we need to introduce a *slack variable* $\zeta^{(i)} \geq 0$ for each instance: $zeta^{(i)}$ measures how much the $i^{th}$ instance is allowed to violate the margin. We now have two conflicting objectives: making the slack variables as small as possible to reduce the margin violations, & making $\frac{1}{2}w^Tw$ as small as possible to increase the margin. This is where the `C` hyperparameter comes in: it allows us to define the tradeoff between two objectives. This gives use the constrained optimisation problem:

$$\begin{split}
\underset{w, b, \zeta}{minimise}\quad \frac{1}{2}w^Tw + C\sum^{m}_{i = 1}\zeta^{(i)}\\
subject\ to \quad t^{(i)}(w^Tx^{(i)} + b) \geq 1 - \zeta^{(i)} \quad and \quad \zeta^{(i)} \geq 0 \quad for\ i = 1, 2, ..., m 
\end{split}$$

## Quadratic Programming

The hard margin & soft margin problems are both convex quadratic optimisation problems with linear constraints. Such problems are known as *Quadratic Programming* (QP) problems. Many off-the-shelf solvers are available to solve QP problems using a variety of techniques that are outside the scope of this book. The general problem formulation is given:

$$\begin{split}
\underset{p}{minimise} \quad \frac{1}{2}p^THp + f^Tp \\
subject\ to \quad Ap \leq b \\
where \quad \Biggl\{\begin{split}
p \quad is\ an\ n_p-dimensional\ vector\ (n_p = number\ of\ parameters), \\
H \quad is\ an\ n_p * n_p\ matrix, \\
f \quad is\ an\ n_p-dimensional\ vector, \\
A \quad is\ an\ n_c * n_p\ matrix\ (n_c = number\ of\ parameters), \\
b \quad is\ an\ n_c-dimensional\ vector
\end{split}
\end{split}$$

Note that the expressions $Ap \leq b$ actually defines $n_c$ constraints: $p^Ta^{(i)} \leq b^{(i)}$ for $i = 1, 2, ..., n_c$, where $a^{(i)}$ is the vector containing the elements of the $i^{th}$ row of $A$ & $b^{(i)}$ is the $i^{(th)}$ element of $b$.

You can easily verify that if you set the QP parameters in the following way, you get the hard margin linear SVM classifier ojective:

* $n_p = n + 1$, where $n$ is the number of features (the +1 is for the bias term).
* $n_c = m$, where $m$ is the number of training instances.
* $H$ is the $n_p * n_p$ identity matrix, except with a zero in the top-left cell (to ignore the bias term).
* $f = 0$, an $n_p$-dimensional vector full of 0s.
* $b = -1$, an $n_c$-dimensional vector full of -1s.
* $a^{(i)} = -t^{(i)}x^{(i)}$, where $x^{(i)}$ is equal to $x^{(i)}$ with an extra bias feature $x_0 = 1$.

So one way to train a hard margin linear SVM classifier is just to use an off-the-shelf QP solver by passint it to the preceding parameters. The resulting vector $p$ will contain the bias term $b = p_0$ & the feature weights $w_i = p_i$ for $i = 1, 2, ..., n$. Similarly, you can use a QP solver to solve the soft margin problem.

However, to use the kernel trick we are going to look at a different constrained optimisation problem.

## The Dual Problem

Given a constrained optimisation problem, know as the *primal problem*, it is possible to express a different but closely related problem, called its *dual problem*. The solution to the dual problem typically gives a lower bound to the solution of the primal problem, but onder some conditions it can even have the smae solutions as the primal problem. Luckily, the SVM problem happens to meet these conditions, so you can choose to solve the primal problem or the dual problem; both will have the same solution. The below equation shows the dual form of the linear SVM objective.

$$\begin{split}
\underset{\alpha}{minimise} \quad \frac{1}{2} \sum^{m}_{i = 1} \sum^{m}_{j = 1} \alpha^{(i)} \alpha^{(j)} t^{(i)} t^{(j)} x^{(i)^T} x^{(j)} - \sum^{m}_{i = 1} \alpha^{(i)} \\
subject\ to\ \quad \alpha^{(i)} \geq 0 \quad for\ i = 1, 2, ..., m
\end{split}$$

Once you find the vector $\hat{\alpha}$ that minimises this equation (using a QP solver), you can compute $\hat{w}$ & $\hat{b}$ that minimise the primal problem using the below equation.

$$\hat{w} = \sum^{m}_{i = 1} \alpha^{(i)} t^{(i)} x^{(i)}$$
$$\hat{b} = \frac{1}{n_s} \underset{\hat{\alpha}^{(i)} > 0}{\sum^{m}_{i = 1}}(t^{(i)} - \hat{w}^Tx^{(i)})$$

The dual problem is faster to solve than the primal when the number of training instances is smaller than the number of features. More importantly, it makes the kernel trick possible, while the primal does not. So what is this kernel trick anyway?

## Kernelised SVM

Suppose you want to apply a $2^{nd}$-degree polynomial transformation to a two-dimensional training set (such as the moons training set), then train a linear SVM classifier on the transformed training set. The below euqation shows the $2^{nd}$-degree polynomial mapping function $\phi$ that you want to apply

$$\phi(x) = 
\phi((\begin{split}
x_1 \\
x_2
\end{split})) = \Biggl(\begin{split} 
x^2_1 \\
\sqrt{2}x_1x_2 \\
x^2_2
\end{split}
\Biggr)$$

Notice that the transformed vector is three-dimensional instead of two-dimensional. Now let's look at what happens to a couple of two-dimensional vectors $a$ & $b$, if we apply this $2^{nd}$ degree polynomial mapping & then compute the dot product of the transformed vectors.

$$$$